<a href="https://colab.research.google.com/github/vannissimo/snake-ga/blob/master/pitch_control_acm_marcature2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
# install findspark using pip
!pip install -q findspark

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
import numpy as np
from bisect import bisect
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import concat,col,lit,desc
from operator import itemgetter
import itertools 
from bisect import bisect
import pandas as pd

In [6]:
team = team_a = 'milan'
opponent = 'sampdoria'
half = 'first half'
half_split = '1'
player_in_possess = 'all'

if opponent == 'cagliari':
  match_name = 'Milan-Cagliari'
  lower_match_name = 'milan_cagliari'
elif opponent == 'sampdoria':
  match_name = 'Sampdoria-Milan'
  lower_match_name = 'sampdoria_milan'

name_path = team +"_"+str(opponent)+"_"+str(half).replace(' ','_')+"_"+str(half_split)+"_"+player_in_possess

name_path

'milan_sampdoria_first_half_1_all'

In [7]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# loading data as parquet
pre_data_players = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/frames_milan_cagliari.parquet").drop('index')
pre_data_players.show(10)

+---------+----------+---------+------+----------+-----------+-------+---------+---------+--------------------+----------+-------------+
|      Vel|         x|        y|  Time|    player|       half|team_id|team_name|player_id|        italian_name|  position|real_position|
+---------+----------+---------+------+----------+-----------+-------+---------+---------+--------------------+----------+-------------+
|      0.0|       0.9|     33.2|5594.5|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
| 2.022165|0.83794284|     32.8|5594.7|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
|2.2488973|       0.8|32.578335|5594.8|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
|2.2148995| 0.7668897|32.359333|5594.9|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
|2.2495675| 0.7331103| 32.13693|5595.0|Ca

In [9]:
pre_pre_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/' + 'pitch_control_' + name_path + '.csv',header='True')
pre_pre_df = pre_pre_df.withColumn('x', pre_pre_df['x'].cast(DoubleType())).withColumn('y', pre_pre_df['y'].cast(DoubleType())).withColumn('target_y', pre_pre_df['target_y'].cast(DoubleType())).withColumn('target_x', pre_pre_df['target_x'].cast(DoubleType()))

pre_pre_df.where(pre_pre_df['target_x'] > -999).where(pre_pre_df['target_y'] < 9999).groupBy('time').agg(func.min('target_x'),func.max('target_x'),func.min('target_y'),func.max('target_y')).show()

AnalysisException: ignored

In [ ]:
#create parameters input boxes (Databricks)

if opponent == 'milan':
  opponent_f = team_a
else:
  opponent_f = opponent

In [ ]:
# FILTRIAMO IL TEMPO DI GIOCO

half_dict = {'first half':'1', 'second half':'2'}

pre_data_players = pre_data_players.withColumnRenamed('Time','time').withColumnRenamed('team_name','team').withColumnRenamed('position','role')

pre_events = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/f24_opta_milan_cagliari.parquet").drop('index')
pre_events_time = pre_events.withColumn('time', 60*pre_events['min']+pre_events['sec'])

max_times = pre_events_time[pre_events_time['period_id'] == half_dict[str(half)]].groupby('period_id').max('time')
min_times = pre_events_time[pre_events_time['period_id'] == half_dict[str(half)]].groupby('period_id').min('time')

min_half_time = min_times.filter(max_times['period_id'] == half_dict[str(half)]).head()['min(time)']
max_half_time = max_times.filter(max_times['period_id'] == half_dict[str(half)]).head()['max(time)']

if half_split == '1':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time) & (pre_data_players['time'] <= min_half_time+float((max_half_time-min_half_time)/2.0)))
  
elif half_split == '2':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time+float((max_half_time-min_half_time)/2.0)) & (pre_data_players['time'] < max_half_time))


In [ ]:
from operator import itemgetter
from pyspark.sql import functions as func
from pyspark.sql.functions import lit

import itertools 
import bisect
import numpy as np
from pyspark.sql.types import DoubleType
import pandas as pd

granularity = 0.5
max_ball_distance = 70.0
max_meters_distance_player_can_receive_ball = 30.0
opponent_lower = opponent.lower()

team_a_goalies = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').select('player').distinct().collect()[0]['player']
opponent_goalies = data_players.where(data_players['team'] == opponent.title()).where(data_players['role'] == 'Goalkeeper').select('player').distinct().collect()[0]['player']

goalie_x = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').groupBy('role').avg('x').rdd.max()[1]

team_a_goalies,opponent_goalies,goalie_x

In [ ]:
min_x = data_players.select('x').rdd.min()[0]
max_x = data_players.select('x').rdd.max()[0]
min_y = data_players.select('y').rdd.min()[0]
max_y = data_players.select('y').rdd.max()[0]

x_lines = list(np.arange(int(min_x)-1,int(max_x)+1,float(max_x-min_x)/6.0))
y_lines = list(np.arange(int(min_y)-1,int(max_y)+1,float(max_y-min_y)/4.0))

mid_x, mid_y = x_lines[3], float(y_lines[2])

if float(np.abs(goalie_x - max_x)) < float(np.abs(goalie_x - min_x)):
  verse = 'right_to_left'
  opponent_verse = 'left_to_right'
else:
  verse = 'left_to_right'
  opponent_verse = 'right_to_left'

team_a_goalies,verse

In [ ]:
pre_pre_df = pre_pre_df.withColumn('time', pre_pre_df['time'].cast(DoubleType()))

In [ ]:
timings = sorted(pre_pre_df.select('time').distinct().rdd.map(lambda a : a[0]).collect())
timings

In [ ]:
pre_events = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/f24_opta_milan_cagliari.parquet").drop('index')
pre_events_time = pre_events.withColumn('time', 60*pre_events['min']+pre_events['sec'])
pre_events_time = pre_events_time.where(pre_events_time['time'].isin(timings))

pre_events_time.show()

In [ ]:
name_in_possess = pre_events_time.select('time','player').distinct()
pre_pre_df_2 = pre_pre_df.where(pre_pre_df['target_x'] > -950).select('time','player_id','x','y','team').distinct()
possess = pre_pre_df_2.join(name_in_possess.withColumnRenamed('player','player_id'),['time','player_id']).select('player_id','team','time','x','y').distinct().withColumnRenamed('player_id','player_in_possess').withColumnRenamed('team','team_in_possess')
possess.show()

In [ ]:
df = pre_pre_df.where(pre_pre_df['target_x'] > -999).distinct()

df.show(10)

In [ ]:
#offside_players = pre_df.where(pre_df['team'] == team_a.capitalize()).select('player_id','time','offside_flag').distinct()

In [ ]:
verse

In [ ]:
df_locs = df.select('player_id','time','team','x','y').distinct()

if verse == 'right_to_left':
  opponent_goal_line = [min_x+5.0, 0.0]  
  team_a_goal_line = [max_x-5.0, 0.0]
else:
  opponent_goal_line = [max_x-5.0, 0.0]
  team_a_goal_line = [min_x+5.0, 0.0]
  
df_locs = df_locs.withColumn('distance_from_opponent_goal_line', 0.25*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2 + (df_locs['y'] - opponent_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2))
df_locs = df_locs.withColumn('distance_from_team_goal_line', 0.25*func.sqrt((df_locs['x'] - team_a_goal_line[0])**2 + (df_locs['y'] - team_a_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - team_a_goal_line[0])**2))

panda_locs = df_locs.toPandas()

verse,opponent_goal_line

In [ ]:
x_ranges = list(np.arange(min_x,max_x,3.0))
y_ranges = list(np.arange(min_y,max_y,3.0))

In [ ]:
def x_area_handling(x):
  
  if x >= 30:
    return x
  elif x < 30 and x >= 10:
    return int(0.8*float(x))
  elif x < 10 and x >= -20:
    return int(0.5*float(x))
  elif x < -20:
    return 10

x_single_area_prob = 100.0/float(len(x_ranges))
y_single_area_prob = 100.0/float(len(y_ranges))

x_ranges_value_right_to_left = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(len(x_ranges)-x_ranges.index(d))), x_ranges))
x_ranges_value_left_to_right = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(x_ranges.index(d))), x_ranges))
y_ranges_value = list(map(lambda d : 100-1.75*(float(y_single_area_prob)*float(np.abs((mid_y-y_ranges.index(d))))), y_ranges))
                      
if verse == 'left_to_right':
  x_ranges_value = x_ranges_value_left_to_right
  opp_x_ranges_value = x_ranges_value_right_to_left
elif verse == 'right_to_left':
  x_ranges_value = x_ranges_value_right_to_left
  opp_x_ranges_value = x_ranges_value_left_to_right
  

In [ ]:
def probabilityPlusMinus(x):
  
  if x[6] == team_a.title():
    return list(x)+[opponent_verse]
  else:
    return list(x)+[verse]
  
def opponentCount(x):
  
  try:
    team_1 = float(x[3])
  except:
    team_1 = 0.0

  try:
    team_2 = float(x[5])
  except:
    team_2 = 0.0

  try:
    opp_1 = int(x[4])
  except:
    opp_1 = 0

  try:
    opp_2 = int(x[6])
  except:
    opp_2 = 0

  if team_1 > team_2:
    return [x[0],x[1],x[2],x[3],-opp_2]
  else:
    return [x[0],x[1],x[2],x[5],-opp_1]

df = df.withColumn('probability', df['probability'].cast(DoubleType()))

pre_max_pitch_pivot_gr = df.where(df['team'] != '').groupBy('target_x','target_y','time').pivot('team').agg(func.max('probability'),func.count('player_id'))

max_pitch_pivot = spark.createDataFrame(pre_max_pitch_pivot_gr.rdd.map(lambda a : opponentCount(a)),['target_x','target_y','time','probability','opponents_count'])

max_pitch_pivot.show()

In [ ]:
max_player = df.select('target_x','target_y','time','player_id','team','probability','distance_from_target','x','y','slope_fuzzy_compare','ball_distance')

In [ ]:
max_pitch_pivot_j = max_pitch_pivot.join(max_player, ['target_x','target_y','time','probability'])

max_pitch_pivot_jj = spark.createDataFrame(max_pitch_pivot_j.rdd.map(lambda f : probabilityPlusMinus(f)),max_pitch_pivot_j.columns+['verse'])

pre_max_pitch_pivot_jjj = max_pitch_pivot_jj.where(max_pitch_pivot_jj['team'] == opponent.capitalize())

pre_max_pitch_pivot_jjj.show()

In [ ]:
max_pitch_pivot_jj.groupBy('player_id','team').avg('probability').show()

In [ ]:
pd_probs = pre_max_pitch_pivot_jjj.toPandas()

In [ ]:
def findBestMaxObs(x,panda,max_obs):
  
  probs = panda[panda['player_id'] == x[0]][panda['time'] == x[1]][['probability','distance_from_target']].values.tolist()
  
  try:
    first_twenty_probs = sorted(list(map(lambda a : a[0],probs)), reverse = True)[:max_obs]
    prob_twentieth = first_twenty_probs[-1]
    distance_twentieth = 100.0

    if float(prob_twentieth) == 100.0:
      first_twenty_distances = sorted(list(map(lambda g : g[1], list(filter(lambda a : a[0] == 100.0,probs)))))[:max_obs]
      distance_twentieth = first_twenty_distances[-1]

    return [prob_twentieth,distance_twentieth]
  except:
    return [0.0,0.0]

players_times_values = spark.createDataFrame(df.where(df['team'] == opponent.title()).select('player_id','time').distinct().rdd.map(lambda a : list(a)+findBestMaxObs(a,pd_probs,60)),['player_id','time','probability_twentieth','distance_twentieth'])

players_times_values.show()

In [ ]:
pd_to_filter = players_times_values.toPandas()

In [ ]:
def obsToFilter(x, panda):
  
  prob_filter, distance_filter = panda[panda['player_id'] == x[5]][panda['time'] == x[2]][['probability_twentieth','distance_twentieth']].values.tolist()[0]
  
  flag = ''
  
  if (float(x[3]) < prob_filter) or (float(x[7]) > distance_filter):
    flag = 'out'
  
  return flag
  
max_pitch_pivot_jjj_pre_filt = spark.createDataFrame(pre_max_pitch_pivot_jjj.rdd.map(lambda a : list(a)+[obsToFilter(a, pd_to_filter)]), pre_max_pitch_pivot_jjj.columns+['obs_flag'])

max_pitch_pivot_jjj = max_pitch_pivot_jjj_pre_filt.where(max_pitch_pivot_jjj_pre_filt['obs_flag'] == '').where(max_pitch_pivot_jjj_pre_filt['probability'] > 0).drop('obs_flag')

In [ ]:
max_pitch_pivot_jjj.show()

In [ ]:
#display(max_pitch_pivot_jjj.where(max_pitch_pivot_jjj['target_x'] == 42.0).where(max_pitch_pivot_jjj['target_y'] == 5.0).where(max_pitch_pivot_jjj['time'] == 1135.0))

In [ ]:
from pyspark.sql.functions import lit

def ifHigherThanLimit(a,limit,maxMinFlag): 
  if maxMinFlag == 'max':
    return float(np.min([limit,a]))
  elif maxMinFlag == 'min':
    return float(np.max([limit,a]))

def addFakeProb(team):
  
  if team == opponent.title():
    return 150.0
  else:
    return -150.0

  
### PITCH ###
pitch_pivot = max_pitch_pivot_jj.select('target_x','target_y','time','probability','player_id','team')
pitch_pivot = pitch_pivot.withColumn('type',lit('pitch'))
pitch_pivot = pitch_pivot.withColumnRenamed('target_x','x').withColumnRenamed('target_y','y')

### PLAYERS ###
players_locs = spark.createDataFrame(data_players.select('player_id','team','time','x','y').withColumn('type',lit('player')).distinct().rdd.map(lambda a : list(a)+[addFakeProb(a[1])]), ['player_id','team','time','x','y','type','probability'])
players_locs = players_locs.where(players_locs['time'].isin(timings))

### BALL ###
ball = players_locs.where(players_locs['player_id'] == 'ball').orderBy('time')
ball_locs = ball.withColumn('probability',lit(200.0)).withColumn('type',lit('ball')).select('x','y','time','probability','player_id','team','type')
pd_ball = ball.toPandas()

In [ ]:
players_ids = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/OPTA/opta_seriea_players.csv", header='True')
players_ids.show()

In [ ]:
player_idddsss = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/frames_milan_cagliari.parquet").drop('index')

pre_player_zones = players_locs.join(player_idddsss.select('player_id','player').distinct(), ['player_id']).drop('player_id')
pre_player_zones = pre_player_zones.withColumnRenamed('player','player_id').withColumn('verse', lit(verse))
pd_zones = pre_player_zones.toPandas()

pre_player_zones.show(66)

In [ ]:
def defineLato(x):
  
  y_value = x[3]

  if verse == 'left_to_right':

    if y_value > 16:
      side = 'Destra'
    elif y_value < -16:
      side = 'Sinistra'
    else:
      side = 'Centrale'
  
  else:
    if y_value < -16:
      side = 'Destra'
    elif y_value > 16:
      side = 'Sinistra'
    else:
      side = 'Centrale'

  return side

player_zones = spark.createDataFrame(pre_player_zones.rdd.map(lambda a : list(a)+[defineLato(a)]), pre_player_zones.columns+['lato'])

player_zones.show(50)

In [ ]:
timings_list = timings#list(map(lambda a : a[0], possess.select('time').distinct().collect()))
timings_list

In [ ]:
player_zones.where(player_zones['team'] == team_a.title()).show(60)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

def findTimeRoles(time_,player_zones_df,var):
  
  player_zones_t = player_zones_df.where(player_zones_df['time'] == time_).where((player_zones_df['player_id'].isin(['Maignan','Radunovic','Audero']))==False)
  vecAssembler = VectorAssembler(inputCols=[var], outputCol="features")
  new_df_t = vecAssembler.transform(player_zones_t)

  kmeans_t = KMeans(k=3, seed=1)  # 3 clusters here
  model_t = kmeans_t.fit(new_df_t.select('features'))
  
  transformed_t = model_t.transform(new_df_t)

  return transformed_t

player_zones_f_team = findTimeRoles(timings_list[0],player_zones.where(player_zones['team'] == team_a.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
player_zones_ff_team = findTimeRoles(timings_list[0],player_zones_f_team.where(player_zones_f_team['team'] == team_a.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')

#player_zones_f_opp = findTimeRoles(timings_list[0],player_zones.where(player_zones['team'] == opponent.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
#player_zones_ff_opp = findTimeRoles(timings_list[0],player_zones_f_opp.where(player_zones_f_opp['team'] == opponent.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')

for j in timings_list[1:]:
  player_zones_fff_team = findTimeRoles(j,player_zones.where(player_zones['team'] == team_a.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
  player_zones_fff_team_2 = findTimeRoles(j,player_zones_fff_team.where(player_zones_fff_team['team'] == team_a.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')
  player_zones_ff_team = player_zones_ff_team.union(player_zones_fff_team_2)
  #player_zones_fff_opp = findTimeRoles(j,player_zones.where(player_zones['team'] == opponent.title()),'x').drop('features').withColumnRenamed('prediction','prediction_x')
  #player_zones_fff_opp_2 = findTimeRoles(j,player_zones_fff_opp.where(player_zones_fff_opp['team'] == opponent.title()),'y').drop('features').withColumnRenamed('prediction','prediction_y')
  #player_zones_ff_opp = player_zones_ff_opp.union(player_zones_fff_opp_2)

In [ ]:
player_zones_f = player_zones_ff_team#.union(player_zones_ff_opp)

In [ ]:
player_zones_f.where(player_zones_f['team'] == team_a.title()).where(player_zones_f['time'] == 289).orderBy('time','prediction_x','prediction_y').show(70)

In [ ]:
player_zones_team = player_zones_f.where(player_zones_f['team'] == team_a.title())

In [ ]:
avg_lines_side_team = player_zones_team.groupBy('time','prediction_x','prediction_y','verse').agg(func.avg('x'),func.avg('y'))

pd_avg_lines_side_team = avg_lines_side_team.toPandas()

In [ ]:
import itertools

def defineLinesFromAvg(time_comp,pd_avg_lines_side_team):

  lines_per_time = []

  panda_gr_x = pd_avg_lines_side_team[pd_avg_lines_side_team['time']== time_comp].groupby(by='prediction_x').mean('avg(x)').sort_values(by = 'avg(x)')
  lines_x = list(panda_gr_x.index)
  avg_x = panda_gr_x['avg(x)'].values.tolist()
  panda_gr_y = pd_avg_lines_side_team[pd_avg_lines_side_team['time']== time_comp].groupby(by='prediction_y').mean('avg(y)').sort_values(by = 'avg(y)')
  lines_y = list(panda_gr_y.index)
  avg_y = panda_gr_y['avg(y)'].values.tolist()

  if verse == 'left_to_right':
    translate_x = {'0':'Linea Difensiva','1':'Seconda Linea','2':'Terza Linea'}
    translate_y = {'0':'Sinistra','1':'Centrale','2':'Destra'}
  else:
    translate_x = {'2':'Linea Difensiva','1':'Seconda Linea','0':'Terza Linea'}
    translate_y = {'0':'Destra','1':'Centrale','2':'Sinistra'}

  for k in lines_y:
    for j in lines_x: 
      lines_per_time.append([time_comp, j, k, avg_x[lines_x.index(j)], translate_x[str(lines_x.index(j))],avg_y[lines_y.index(k)], translate_y[str(lines_y.index(k))]])
    
  return lines_per_time

In [ ]:
sparkContext=spark.sparkContext
timings_rdd = sparkContext.parallelize(timings)

In [ ]:
lines_definition_per_time = spark.createDataFrame(timings_rdd.flatMap(lambda c : defineLinesFromAvg(c,pd_avg_lines_side_team)), ['time','prediction_x','prediction_y','avg_x','linea','avg_y','lato_new'])

In [ ]:
pre_player_zones_team_final = player_zones_team.drop('lato').join(lines_definition_per_time,['time','prediction_x','prediction_y'])
pre_player_zones_team_final = pre_player_zones_team_final.withColumnRenamed('linea','linea_di_pressione')
player_zones_team_final = pre_player_zones_team_final.withColumnRenamed('lato_new','lato').select('time','linea_di_pressione','lato','avg_x','avg_y').distinct()

In [ ]:
pre_player_zones_team_final.show()

In [ ]:
all_team_lines_and_side = pre_player_zones_team_final.toPandas()

In [ ]:
pd_avg_lines_pivot = player_zones_team_final.toPandas()

In [ ]:
'''
def func_y_value(y):
  
  if y > 11:
    return 'Sinistra'
  elif y < -11:
    return 'Destra'
  else:
    return 'Centrale'
  
def defineLine(t,pre_panda,verse_t,translate_dict,asse,index):
  
  if asse == 'x':
    var = 'prediction_y'
  else:
    var = 'prediction_x'

  out = []

  for i in [0,1,2]:
    try:
      pre_panda_f = pre_panda[pre_panda['time'] == t][pre_panda[var] == i].values.tolist()
      
      if verse_t == 'right_to_left':
        pre_panda_f_sort = sorted(pre_panda_f, key = itemgetter(index), reverse = True)
      else:
        pre_panda_f_sort = sorted(pre_panda_f, key = itemgetter(index))
      
      if asse == 'x':
        pre_panda_f_sort_defined = list(map(lambda c : list(c)+[translate_dict[str(pre_panda_f_sort.index(c))]],pre_panda_f_sort))
      else:
        pre_panda_f_sort_defined = list(map(lambda c : list(c)+[func_y_value(c[index])],pre_panda_f_sort))

      out = out + pre_panda_f_sort_defined
    except:
      pass

  
  return out

translate_x = {'0':'Linea Difensiva','1':'Seconda Linea', '2': 'Terza Linea'}
translate_y = {'0':'Sinistra','1':'Centrale', '2': 'Destra'}

x_index = avg_lines_side_team.columns.index('avg(x)')
y_index = avg_lines_side_team.columns.index('avg(y)')

df_avg_lines_x = spark.createDataFrame(timings_rdd.flatMap(lambda a : defineLine(a,pd_avg_lines_side_team,verse,translate_x,'x',x_index)),['time','cluster_x','cluster_y','verse','avg_x','avg_y','linea_di_pressione'])
df_avg_lines_y = spark.createDataFrame(timings_rdd.flatMap(lambda b : defineLine(b,pd_avg_lines_side_team,verse,translate_y,'y',y_index)),['time','cluster_x','cluster_y','verse','avg_x','avg_y','lato'])

df_avg_lines = df_avg_lines_x.join(df_avg_lines_y.drop('avg_x').drop('avg_y').drop('verse'), ['time','cluster_x','cluster_y'])

df_avg_lines = df_avg_lines.withColumn('avg_y',df_avg_lines['avg_y'].cast(DoubleType())).withColumn('avg_x',df_avg_lines['avg_x'].cast(DoubleType())).orderBy('time','linea_di_pressione','lato')
df_avg_lines = df_avg_lines.withColumn('linea_lato', concat(col('linea_di_pressione'),lit(' '),col('lato')))
df_avg_lines_pivot = df_avg_lines.groupBy('time','verse','lato','linea_di_pressione').agg(func.avg('avg_x'),func.avg('avg_y')).withColumnRenamed('avg(avg_x)','avg_x').withColumnRenamed('avg(avg_y)','avg_y')

df_avg_lines_pivot.show()

pd_avg_lines_pivot = df_avg_lines_pivot.toPandas()
'''

In [ ]:
pd_player_zones = player_zones.toPandas()
pd_player_zones

In [ ]:
player_index = player_zones.columns.index('player_id')
x_index = player_zones.columns.index('x')
y_index = player_zones.columns.index('y')
team_index = player_zones.columns.index('team')
verse_index = player_zones.columns.index('verse') 
time_index = player_zones.columns.index('time') 

player_zones.columns, team_index

In [ ]:
from numpy.core.function_base import geomspace
def definePositionBetweenLines(x):
  
  player = x[player_index]
  x_value = x[x_index]
  y_value = x[y_index]
  team = x[team_index]
  verse = x[verse_index]
  time = x[time_index]

  pd_time = pd_avg_lines_pivot[pd_avg_lines_pivot['time'] == time]
  pd_player_time = pd_player_zones[pd_player_zones['time'] == time][pd_player_zones['team'] == team_a.title()][pd_player_zones['player_id'] != 'Maignan']

  #terza_linea = float(x[-1])
  
  if verse == 'right_to_left': 
    destra = -33.0
    sinistra = 33.0
    terza_linea = -52.0
  else:
    destra = 33.0
    sinistra = -33.0
    terza_linea = 52.0

  centrale = float(np.mean(pd_time[pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))
  if len(pd_time[pd_time['lato'] == 'Destra']) > 0:
    destra = float(np.mean(pd_time[pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
  if len(pd_time[pd_time['lato'] == 'Sinistra']) > 0:
    sinistra = float(np.mean(pd_time[pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
  
  linea_difensiva = float(np.mean(pd_time[pd_time['linea_di_pressione'] == 'Linea Difensiva']['avg_x'].values.tolist()))
  seconda_linea = float(np.mean(pd_time[pd_time['linea_di_pressione'] == 'Seconda Linea']['avg_x'].values.tolist()))
  if len(pd_time[pd_time['linea_di_pressione'] == 'Terza Linea']) > 0:
    terza_linea = float(np.mean(pd_time[pd_time['linea_di_pressione'] == 'Terza Linea']['avg_x'].values.tolist()))
  else:
    if verse == 'right_to_left':
      most_advanced_player_x = float(np.min(pd_player_time['x'].values.tolist()))
    else:
      most_advanced_player_x = float(np.max(pd_player_time['x'].values.tolist()))
    
    terza_linea = most_advanced_player_x
  
  list_lines = list(map(lambda a : float(np.round(a,1)), [linea_difensiva, seconda_linea,terza_linea]))
  list_lines_lato = list(map(lambda a : float(np.round(a,1)), [centrale,destra,sinistra]))
 
  if team == opponent.title():

    if verse == 'right_to_left':
      
      if x_value >= seconda_linea and x_value <= linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Tra le linee'

      elif x_value > linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))
       
        posizione = 'Profondità'
      
      elif x_value > seconda_linea-6.0 and x_value < seconda_linea:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Possibile Inserimento'

      else:
        posizione = ''
      
      try:
        if y_value < destro:
          lato = 'Ampiezza Destra'
        elif y_value > sinistro:
          lato =  'Ampiezza Sinistra'
        elif y_value <= sinistro and y_value > centrale:
          lato =  'Centro-Sinistra'
        elif y_value >= destro and y_value <= centrale:
          lato =  'Centro-Destra'
        else:
          lato = ''
      except:
        lato = ''
        
    else:
        
      if x_value < seconda_linea and x_value > linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Tra le Linee'
      elif x_value < linea_difensiva:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))
       
        posizione = 'Profondità'

      elif x_value < seconda_linea+6.0 and x_value > seconda_linea:
        destro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Destra']['avg_y'].values.tolist()))
        sinistro = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Sinistra']['avg_y'].values.tolist()))
        centrale = float(np.mean(pd_time[pd_time['linea_di_pressione'].isin(['Linea Difensiva','Seconda Linea'])][pd_time['lato'] == 'Centrale']['avg_y'].values.tolist()))

        posizione = 'Possibile Inserimento'

      else:
        posizione = ''

      try:
        if y_value > destro:
          lato = 'Ampiezza Destra'
        elif y_value < sinistro:
          lato =  'Ampiezza Sinistra'
        elif y_value >= sinistro and y_value < centrale:
          lato =  'Centro-Sinistra'
        elif y_value <= destro and y_value >= centrale:
          lato =  'Centro-Destra'
        else:
          lato = ''
      except:
        lato = ''

  else:
    if player != 'Maignan':
      posizione,lato = all_team_lines_and_side[all_team_lines_and_side['player_id'] == player][all_team_lines_and_side['time'] == time][['linea_di_pressione','lato_new']].values.tolist()[0]
      if verse == 'right_to_left':
        if (x_value >= linea_difensiva-6.0 and x_value < linea_difensiva-1.0) and posizione == "Seconda Linea":
          posizione = 'Seconda Linea - Schermo'
      else:
        if (x_value <= linea_difensiva+6.0 and x_value > linea_difensiva+1.0) and posizione == "Seconda Linea":
          posizione = 'Seconda Linea - Schermo'
  
    else:
      posizione = ''
      lato = ''
  

  out = list(x)+[posizione,lato,list_lines,list_lines_lato]
  return out
  
positioned_players_ = spark.createDataFrame(player_zones.rdd.map(lambda a : definePositionBetweenLines(a)), player_zones.columns+['posizione_x','lato_new','values_x','values_y'])
positioned_players_.where(positioned_players_['time'].isin([4315])).orderBy(['time','team','posizione_x','lato']).show(50)

In [ ]:
positioned_players_r = positioned_players_.drop('lato_new').drop('values_x').drop('values_y')
positioned_players_rr = positioned_players_r#.withColumnRenamed('lato_new','lato')

In [ ]:
positioned_players_out = positioned_players_rr.drop('type')

In [ ]:
positioned_players_out.orderBy('time','team','posizione_x','lato').show(22)

In [ ]:
def addNegativeProb(x):

  team = x[team_index] 
  probability = x[prob_index]

  if team == team_a.title(): 
    new_prob = -float(probability) 
  else: 
    new_prob = float(probability)

  y = list(x[:prob_index])+[new_prob]+list(x[1+prob_index:])

  return y

pitch_pivot_union = pitch_pivot.distinct()

team_index, prob_index = pitch_pivot_union.columns.index('team'), pitch_pivot_union.columns.index('probability')

pitch_pivot_union = spark.createDataFrame(pitch_pivot_union.rdd.map(lambda y : addNegativeProb(y)), pitch_pivot_union.columns)

for i in players_locs.columns: 
  if 'class' in str(i): 
    pitch_pivot_union = pitch_pivot_union.withColumn(str(i), lit(''))

out = pitch_pivot_union.union(players_locs.select(pitch_pivot_union.columns)).union(ball_locs.select(pitch_pivot_union.columns)).withColumnRenamed('player_id','player')  

In [ ]:
pd_pitch = max_pitch_pivot_jj.toPandas()

In [ ]:
panda_possess = possess.toPandas()
panda_possess_x_y = out.where(out.team != team_a.title()).where(out.team != opponent.title()).toPandas()

In [ ]:
from bisect import bisect
from pyspark.sql.window import Window

def areaDanger(d,x_ranges,y_ranges):
  
  if d[verse_index] == 'left_to_right':
    x_values = x_ranges_value_right_to_left
  else:
    x_values = x_ranges_value_left_to_right
  
  try:
    x_value = x_values[bisect(x_ranges,d[x_index])]
  except:
    try:
      x_value = x_values[bisect(x_ranges,d[x_index])-1]
    except:
      x_value = x_values[bisect(x_ranges,d[x_index])+1]
      
  try:
    y_value = y_ranges_value[bisect(y_ranges,d[y_index])]
  except:
    try:
      y_value = y_ranges_value[bisect(y_ranges,d[y_index])-1]
    except:
      y_value = y_ranges_value[bisect(y_ranges,d[y_index])+1]
  
  if x_value < 50:
    y_value = 10
  
  return [bisect(x_ranges,d[x_index]),bisect(y_ranges,d[y_index]),float(np.round(x_value,0)),float(np.round(y_value,0)),float(np.round(x_value+y_value,0))]
 
def findOpponentsOvercomeByThePass(x,panda,panda_possess,pd_pitch):
  
  player_x = x[x_index]
  player_y = x[y_index]
  verse = x[verse_index]
  time_ = x[time_index]
  
  player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0]
  possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['x','y']].values.tolist()[0]
  
  sit_x = player_x
  sit_y = player_y

  x_advancement_flag = float(np.sign(team_a_goal_line[0] - player_x))

  values_x = [sit_x-(x_advancement_flag*10),sit_x+(x_advancement_flag*10)]

  back_min_x,ahead_max_x = float(np.min(values_x)),float(np.max(values_x))

  up_y = sit_y+10
  down_y = sit_y-10

  quarters_control_percentages = []

  close_opponents_panda = panda[panda['team'] == team_a.title()][panda['time'] == time_][panda['x'] > player_x-4.0][panda['x'] < player_x+4.0][panda['y'] > player_y-4.0][panda['x'] < player_y+4.0]
  close_opponents = len(close_opponents_panda)
  opponents_x_y = panda[panda['team'] == team_a.title()][panda['time'] == time_][panda['player_id'] != team_a_goalies][['x','y']].values.tolist()
  
  #player_distance_from_goal_line = float(np.sqrt((player_x - team_a_goal_line[0])**2 + (player_y - team_a_goal_line[1])**2))
  #possess_distance_from_goal_line = float(np.sqrt((possess_x - team_a_goal_line[0])**2 + (possess_y - team_a_goal_line[1])**2))

  if verse == 'left_to_right':
    player_x_distance_from_goal_line = float(player_x - team_a_goal_line[0])
    possess_x_distance_from_goal_line = float(possess_x - team_a_goal_line[0])
  else:
    player_x_distance_from_goal_line = -float(player_x - team_a_goal_line[0])
    possess_x_distance_from_goal_line = -float(possess_x - team_a_goal_line[0])
  
  team_advancement_x = possess_x_distance_from_goal_line - player_x_distance_from_goal_line
  ball_distance = float(np.sqrt((player_x-possess_x)**2+(player_y-possess_y)**2))

  #opponents_distances_to_goal_line = []
  opponents_distances_to_goal_line_x = []

  for j in opponents_x_y:
    if verse == 'left_to_right':
      dist = -float(team_a_goal_line[0]-j[0])
    else:
      dist = float(team_a_goal_line[0]-j[0])

    #opponents_distances_to_goal_line.append(float(np.sqrt((team_a_goal_line[0]-j[0])**2+(team_a_goal_line[1]-j[1])**2)))
    opponents_distances_to_goal_line_x.append(dist)
    
  #opponents_from_goal_line = len(list(filter(lambda a : a < player_distance_from_goal_line-1.0, opponents_distances_to_goal_line)))
  #opponents_from_possess_to_goal_line = len(list(filter(lambda a : a < possess_distance_from_goal_line-1.0, opponents_distances_to_goal_line)))

  opponents_from_goal_line_x = len(list(filter(lambda a : a < player_x_distance_from_goal_line-4.0, opponents_distances_to_goal_line_x)))
  opponents_from_possess_to_goal_line_x = len(list(filter(lambda a : a < possess_x_distance_from_goal_line-1.0, opponents_distances_to_goal_line_x)))

  #opponents_cut = opponents_from_possess_to_goal_line-opponents_from_goal_line
  opponents_cut_x = opponents_from_possess_to_goal_line_x-opponents_from_goal_line_x

  #return [closest_opponent_distance,opponents_from_goal_line,close_opponents,player_distance_from_goal_line,opponents_cut]
  return [opponents_from_possess_to_goal_line_x,close_opponents,player_x_distance_from_goal_line,opponents_cut_x,team_advancement_x,ball_distance]
  
pre_kpis = max_pitch_pivot_jjj.where((max_pitch_pivot_jjj['player_id'].isin([team_a_goalies])) == False).select('player_id','team','verse','time','x','y').distinct()

verse_index = pre_kpis.columns.index('verse')
time_index = pre_kpis.columns.index('time')
y_index = pre_kpis.columns.index('y')
x_index = pre_kpis.columns.index('x')

pre_kpis = pre_kpis.withColumn('x', pre_kpis['x'].cast(DoubleType())).withColumn('y', pre_kpis['y'].cast(DoubleType()))

ppp_area_targets = spark.createDataFrame(pre_kpis.rdd.map(lambda g : list(g)+areaDanger(g,x_ranges,y_ranges)+findOpponentsOvercomeByThePass(g,panda_locs,panda_possess,pd_pitch)), pre_kpis.columns+['x_area','y_area','x_value','y_value','area_value','possess_opponents_from_goal_line','close_opponents','player_distance_from_goal_line','opponents_cut','team_advancement','ball_distance'])


In [ ]:
area_targets_advance_pre_final = ppp_area_targets.select('player_id','time','team','area_value','ball_distance','possess_opponents_from_goal_line','x','y','close_opponents','player_distance_from_goal_line','opponents_cut','team_advancement','verse').distinct()

area_targets_advance_pre_final.show()

In [ ]:
def singlePointOpponents(x,y,panda_locs,opponent,time_,meters_to_intercept,possess_x,possess_y,fromPossessOrToGoal):
  
  distance = distance = float(np.sqrt(((x-possess_x)**2)+((y-possess_y)**2)))
  opps =  panda_locs[panda_locs['team'] == team_a.title()][panda_locs['player_id'] != 'Maignan'] #automatizzare -> team_a_goalies è in codice (p123...)

  if fromPossessOrToGoal == 'possess':
    opponent_meters_run = 3.0*(float(distance)/20.0)
    on_trajectory = opps[opps['time'] == time_][opps['x'] > x-meters_to_intercept-opponent_meters_run][opps['x'] < x+meters_to_intercept+opponent_meters_run][opps['y'] > y-meters_to_intercept-opponent_meters_run][opps['y'] < y+meters_to_intercept+opponent_meters_run]['player_id'].values.tolist()
 
  else:
    players_opps = opps['player_id'].values.tolist()
    
    on_trajectory = []

    for p in players_opps:
      player_x,player_y = opps[opps['player_id'] == p][['x','y']].values.tolist()[0]
      player_distance = float(np.sqrt(((x-player_x)**2)+((y-player_y)**2)))
      if distance > 30:
        if player_distance < distance-4.0:
          on_trajectory.append(p)
      else:
        if player_distance < distance-0.5:
          on_trajectory.append(p)


  return on_trajectory

def interceptionOpponentsFix(player,time_,target_x,target_y,verse,panda,panda_possess,fromPossessOrToGoal):
  
  if fromPossessOrToGoal == 'possess':
    player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0]
    possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['x','y']].values.tolist()[0]
  else:
    player_possess = panda_possess[panda_possess['time'] == time_]['player_in_possess'].values.tolist()[0][0]
    possess_x,possess_y = team_a_goal_line[0], team_a_goal_line[1]
  
  if player == player_possess:
    possess_y = possess_y+0.3
    possess_x = possess_x+0.3

  distance = float(np.sqrt(((target_x-possess_x)**2)+((target_y-possess_y)**2)))
  m = (possess_y-target_y)/(possess_x-target_x)
  b = (possess_x*target_y - target_x*possess_y)/(possess_x-target_x)

  if possess_x > target_x:
    xs = sorted(list(np.arange(target_x,possess_x,float(possess_x-target_x)/distance)), reverse = True)
  else:
    xs = list(np.arange(possess_x,target_x,float(target_x-possess_x)/distance))

  pre_trajectory = list(map(lambda xx: [float(xx),float((m*xx)+b)], xs))

  if fromPossessOrToGoal == 'possess':
    if distance > 30:
      trajectory = pre_trajectory[1:11]
      interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,possess_x,possess_y,fromPossessOrToGoal), trajectory))))
      interception_opponents_no_duplicates = len(list(dict.fromkeys(interception_opponents)))
    else:
      trajectory = pre_trajectory#+[pre_trajectory[-5],pre_trajectory[-4],pre_trajectory[-3],pre_trajectory[-2],pre_trajectory[-1]]
      interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,possess_x,possess_y,fromPossessOrToGoal), trajectory))))
      interception_opponents_no_duplicates = len(list(dict.fromkeys(interception_opponents)))
  else:
    trajectory = pre_trajectory[1:-10]
    interception_opponents = list(itertools.chain.from_iterable(list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,2.0,target_x,target_y,fromPossessOrToGoal), trajectory))))
    interception_opponents_no_duplicates = len(list(dict.fromkeys(interception_opponents)))

  if str(player) == str(player_possess):
    possessFlag = "Sul portatore"
  else:
    possessFlag = ""
  
  pre_interception_opponents = [interception_opponents_no_duplicates]
  
  if fromPossessOrToGoal == 'possess':
    close_opponents_distances = []
    if verse == 'right_to_left':
      opps_data = panda[panda['time'] == time_][panda['team'] == team_a.title()][panda['x'] > target_x-2.0][['player_id','x','y']].values.tolist()
    else:
      opps_data = panda[panda['time'] == time_][panda['team'] == team_a.title()][panda['x'] < target_x+2.0][['player_id','x','y']].values.tolist()
  
    for j in opps_data:
     close_opponents_distances.append([j[0],float(np.sqrt((j[1]-target_x)**2+(j[2]-target_y)**2))])
    
    closest_opponent_player, closest_opponent_distance = sorted(close_opponents_distances, key=itemgetter(1))[0]
  
    aa_interception_opponents = pre_interception_opponents+[possess_x,possess_y,possessFlag,closest_opponent_distance,closest_opponent_player,' '.join(list(dict.fromkeys(interception_opponents)))]
  else:
    aa_interception_opponents = pre_interception_opponents+[' '.join(list(dict.fromkeys(list(map(lambda a : str(a[:3]),interception_opponents)))))]
  
  return aa_interception_opponents

In [ ]:
interception_opps = spark.createDataFrame(area_targets_advance_pre_final.select('player_id','time','x','y','verse').distinct().rdd.map(lambda g : list(g)+list(interceptionOpponentsFix(g[0],g[1],g[2],g[3],g[4],panda_locs,panda_possess,'possess'))+list(interceptionOpponentsFix(g[0],g[1],g[2],g[3],g[4],panda_locs,panda_possess,'goal'))), ['player_id','time','x','y','verse','interception_opponents','possess_goal_x','possess_goal_y','tipo_di_marcatura','closest_opponent_distance','closest_opponent_player','interception_players','opponents_to_goal_line','to_goal_line_players'])

In [ ]:
interception_opps.show()

In [ ]:
#count e probability

count_and_probs = pitch_pivot_union.groupBy('time','player_id').agg(func.count('time'),func.avg('probability')).withColumnRenamed('count(time)','Area (Mq)').withColumnRenamed('avg(probability)','Probabilità di Ricezione')
count_and_probs.where(count_and_probs['time'].isin([4315])).orderBy('time').show(50)

In [ ]:
duelli = positioned_players_out.join(count_and_probs,['time','player_id'])

In [ ]:
duelli.where(duelli['time'] == 4315).orderBy('team','posizione_x','lato').show()

In [ ]:
kpis = interception_opps.join(count_and_probs, ['time','player_id'])
kpis.show()

In [ ]:
kpis_out = area_targets_advance_pre_final.drop('x').drop('y').join(kpis.drop('verse'),['time','player_id'])
kpis_outtt = kpis_out.drop('_18')#.join(timed_roles, ['time','player_id'])

kpis_outtt.show()

In [ ]:
ids = players_ids.select('player_id','last_name').withColumnRenamed('player_id','player').distinct()
pd_ids = ids.toPandas()

In [ ]:
def addingPlayerName(x,panda):
  
  try:
    last_name = panda[panda['player'] == x[new_player_index]]['last_name'].values.tolist()[0]
    return last_name
  except:
    return x[new_player_index]

new_player_index = out.columns.index('player')

out_player = spark.createDataFrame(out.where(out['team'] != 'ball').where(out['type'] == 'player').distinct().rdd.map(lambda a : list(a)+[addingPlayerName(a,pd_ids)]), out.columns+['last_name'])
out_player_f = out_player.withColumnRenamed('player','player_code')
out_player_f1 = out_player_f.withColumnRenamed('last_name','player')
out_player_ids = out_player_f1.select(out.columns)

out_pitch = out.where(out['type'] == 'pitch')

out_player_ff_pre_ball = out_player_ids.union(out_pitch)

In [ ]:
pre_ball_out = spark.createDataFrame(panda_possess)
pre_ball_out2 = pre_ball_out.withColumnRenamed('team_in_possess','team').withColumnRenamed('player_in_possess','player').withColumn('type',lit('ball')).withColumn('probability',lit(-150.0))
ball_out = pre_ball_out2.select(out_player_ff_pre_ball.columns)

pitch_out = out_player_ff_pre_ball.union(ball_out)

In [ ]:
pitch_out.groupBy('time','type','team').count().orderBy('time').show(100)

In [ ]:
'''
# area_value
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('area_value'),func.max('area_value')).withColumnRenamed('avg(area_value)','avg_area_value').withColumnRenamed('max(area_value)','max').withColumn('kpi', lit('area_value')).fillna(0.0)
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final_area_value.withColumn('area_value_index', (0.45*area_targets_advance_pre_final_area_value['avg_area_value']+0.55*area_targets_advance_pre_final_area_value['max'])).select('player_id','time','avg_area_value','area_value_index')
# opponents_cut
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('opponents_cut'),func.max('opponents_cut')).withColumnRenamed('avg(opponents_cut)','avg_opponents_cut').withColumnRenamed('max(opponents_cut)','max').withColumn('kpi', lit('opponents_cut')).fillna(0.0)
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final_opponents_cut.withColumn('opponents_cut_index', (0.45*area_targets_advance_pre_final_opponents_cut['avg_opponents_cut']+0.55*area_targets_advance_pre_final_opponents_cut['max'])).select('player_id','time','avg_opponents_cut','opponents_cut_index')

# percentage_zone_in_control
#area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('percentage_zone_in_control'),func.max('percentage_zone_in_control')).withColumnRenamed('avg(percentage_zone_in_control)','avg').withColumnRenamed('max(percentage_zone_in_control)','max').withColumn('kpi', lit('percentage_zone_in_control'))
#area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final_percentage_zone_in_control.withColumn('percentage_zone_in_control_index', func.log((1+(0.45*area_targets_advance_pre_final_percentage_zone_in_control['avg']+0.55*area_targets_advance_pre_final_percentage_zone_in_control['max']))/area_targets_advance_pre_final_percentage_zone_in_control['avg'])).select('player_id','time','percentage_zone_in_control_index')

# team_advancement
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('team_advancement'),func.max('team_advancement')).withColumnRenamed('avg(team_advancement)','avg_team_advancement').withColumnRenamed('max(team_advancement)','max').withColumn('kpi', lit('team_advancement')).fillna(0.0)
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final_team_advancement.withColumn('team_advancement_index', (0.45*area_targets_advance_pre_final_team_advancement['avg_team_advancement']+0.55*area_targets_advance_pre_final_team_advancement['max'])).select('player_id','time','avg_team_advancement','team_advancement_index')
# distance_from_target
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('distance_from_target'),func.max('distance_from_target')).withColumnRenamed('avg(distance_from_target)','avg_distance_from_target').withColumnRenamed('max(distance_from_target)','max').withColumn('kpi', lit('distance_from_target')).fillna(0.0)
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final_distance_from_target.withColumn('distance_from_target_index', (0.45*area_targets_advance_pre_final_distance_from_target['avg_distance_from_target']+0.55*area_targets_advance_pre_final_distance_from_target['max'])).select('player_id','time','distance_from_target_index')
# probability
area_targets_advance_pre_final_probability = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('probability'),func.max('probability')).withColumnRenamed('avg(probability)','avg_probability').withColumnRenamed('max(probability)','max').withColumn('kpi', lit('probability')).fillna(0.0)
area_targets_advance_pre_final_probability = area_targets_advance_pre_final_probability.withColumn('probability_index', (0.45*area_targets_advance_pre_final_probability['avg_probability']+0.55*area_targets_advance_pre_final_probability['max'])).select('player_id','time','avg_probability','probability_index')
# ball_distance
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('ball_distance'),func.max('ball_distance')).withColumnRenamed('avg(ball_distance)','avg_ball_distance').withColumnRenamed('max(ball_distance)','max').withColumn('kpi', lit('ball_distance')).fillna(0.0)
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final_ball_distance.withColumn('ball_distance_index', (0.45*area_targets_advance_pre_final_ball_distance['avg_ball_distance']+0.55*area_targets_advance_pre_final_ball_distance['max'])).select('player_id','time','avg_ball_distance','ball_distance_index')

count = area_targets_advance_pre_final.groupBy('player_id','time').count()

kpis = area_targets_advance_pre_final_area_value.join(count,['player_id','time']).join(area_targets_advance_pre_final_opponents_cut, ['player_id','time']).join(area_targets_advance_pre_final_team_advancement, ['player_id','time']).join(area_targets_advance_pre_final_distance_from_target, ['player_id','time']).join(area_targets_advance_pre_final_probability, ['player_id','time']).join(area_targets_advance_pre_final_ball_distance, ['player_id','time'])
'''

In [ ]:
duelli_path = 'duelli_'+name_path
duelli_path

In [ ]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+duelli_path+".csv"

duelli.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)

In [ ]:
new_path = 'pitch_control_2_'+name_path
new_path

In [ ]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+new_path+".csv"

pitch_out.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)

In [ ]:
new_path_k = 'kpis_new_'+name_path
new_path_k

In [ ]:
temp_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/__temp_k"
target_path = "/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/"+new_path_k+".csv"

kpis_out.coalesce(1).write.mode("overwrite").option('delimiter',',').option('header','true').csv(temp_path)

Path = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path

# get the part file generated by spark write
fs = Path(temp_path).getFileSystem(spark.sparkContext._jsc.hadoopConfiguration())
csv_part_file = fs.globStatus(Path(temp_path + "/part*"))[0].getPath()

# move and rename the file
fs.rename(csv_part_file, Path(target_path))
fs.delete(Path(temp_path), True)